In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import re
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score
from sklearn.metrics import (
    classification_report as creport
)
from google.colab import files
uploaded= files.upload ()

Saving BinaryPublicFigures.xlsx to BinaryPublicFigures (1).xlsx


### **Load the Ar_PuFi Binary Datset**

In [ ]:
import io
df= pd.read_excel(io.BytesIO(uploaded['Ar-PuFi.xlsx']))
df= df[['Text','Label']]
df

In [ ]:
df["Label"] = df["Label"].astype(str).astype(int)
dataTypeObj = df.dtypes['Label']
print(dataTypeObj)

In [ ]:
!git clone https://github.com/aub-mind/arabert.git


fatal: destination path 'arabert' already exists and is not an empty directory.


In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q farasapy
!pip install -q pyarabic
!pip install -q emoji

In [ ]:
import pandas as pd 

from sklearn.model_selection import train_test_split

from pathlib import Path

import torch
from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoModel, BertTokenizerFast, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(df['Text'], df['Label'], random_state=42, 
                                                                    test_size=0.3)

val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, random_state=42, 
                                                                test_size=0.5)

len(train_texts), len(val_texts), len(test_texts)

(17147, 3674, 3675)

In [ ]:
BERT_MODEL_NAME = 'aubmindlab/bert-base-arabertv02-twitter'
bert = AutoModel.from_pretrained(BERT_MODEL_NAME)

tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL_NAME)

In [ ]:
# get length of all the messages in the train set
seq_len = [len(tokenizer.encode(i)) for i in train_texts]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
max_seq_len = 40
model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME, num_labels=2)


Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmi

In [ ]:
train_encodings = tokenizer(train_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
val_encodings = tokenizer(val_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)

In [ ]:
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.to_list()

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
train_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]),
 'input_ids': tensor([    2,  8808,   306,   759,  1793,   305, 12470,   516,   228, 10524,
         19372,   195, 11135,  8524,   421,  4904,   305,  7084, 17765,  3074,
          1072,  7084, 51345, 27130, 27025,  5510,  7155,   186,  1730, 24974,
          3837,   209,  1955,   740,  4548, 43565,     3,     0,     0,     0]),
 'labels': tensor(1),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# function to pass to out trainer in order to compute accuracy, f1 score, precision and recall
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

### **Ar-PuFi Training**

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_strategy='epoch',
    evaluation_strategy='epoch'
    )

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 3675
  Batch size = 64


PredictionOutput(predictions=array([[-2.374717 ,  2.2295437],
       [-1.2559986,  1.2673159],
       [ 2.7836356, -2.477317 ],
       ...,
       [ 2.7058592, -2.642757 ],
       [-1.6745683,  2.1411371],
       [ 1.1630284, -1.001087 ]], dtype=float32), label_ids=array([1, 1, 0, ..., 0, 1, 0]), metrics={'test_loss': 0.1944262534379959, 'test_accuracy': 0.9355102040816327, 'test_f1': 0.9354523743795635, 'test_precision': 0.9352888888888888, 'test_recall': 0.9357860077421651, 'test_runtime': 496.2921, 'test_samples_per_second': 7.405, 'test_steps_per_second': 0.117})

### **Mubarak_17 Testing**

In [ ]:
df_test= pd.read_csv('/content/Mubarak_17.csv')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

1    3820
0    2204
Name: Label, dtype: int64

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 6024
  Batch size = 64


PredictionOutput(predictions=array([[ 0.93654454, -0.78370845],
       [ 1.5639944 , -1.188667  ],
       [ 1.0169153 , -0.59422064],
       ...,
       [-1.2732489 ,  1.6433228 ],
       [ 2.3662424 , -2.320475  ],
       [ 0.716555  , -0.758003  ]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 0, 1]), metrics={'test_loss': 1.4642623662948608, 'test_accuracy': 0.5341965471447543, 'test_f1': 0.525182644920701, 'test_precision': 0.652132225196037, 'test_recall': 0.6154478767780618, 'test_runtime': 801.4687, 'test_samples_per_second': 7.516, 'test_steps_per_second': 0.119})

### **Alakrot Testing**

In [ ]:
import io
df_test= pd.read_csv('/content/Alakrot.csv')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

In [ ]:
df_test['Label'] = df_test['Label'].replace([2],[0])
df_test['Label'].value_counts()

# frames= [Writer,Sport, Judiciary, Politics, Art, Business]
# df= pd.concat(frames)

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 5846
  Batch size = 64


PredictionOutput(predictions=array([[ 0.47458476, -0.14468701],
       [-1.5225309 ,  1.7674847 ],
       [ 0.26915416, -0.30794743],
       ...,
       [-0.6067045 ,  0.70254457],
       [ 2.3676991 , -2.2648644 ],
       [ 0.2673576 ,  0.13823032]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 0, 1]), metrics={'test_loss': 1.0759153366088867, 'test_accuracy': 0.611871365035922, 'test_f1': 0.6089533081740581, 'test_precision': 0.6997768322562785, 'test_recall': 0.675119519924146, 'test_runtime': 796.3687, 'test_samples_per_second': 7.341, 'test_steps_per_second': 0.116})

### **Albadi Testing** 

In [ ]:
import io
df_test= pd.read_csv('/content/Albadi.csv')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

1    2654
0    2586
Name: Label, dtype: int64

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

### **L_HSAB Testing** 

In [ ]:
import io
df_test= pd.read_csv('/content/L_HSAB.csv')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 11268
  Batch size = 64


PredictionOutput(predictions=array([[-1.9666207 ,  2.1141243 ],
       [-1.7103305 ,  1.8891842 ],
       [-0.88532555,  1.1762778 ],
       ...,
       [ 2.3952448 , -2.0035844 ],
       [ 2.1653888 , -1.9863443 ],
       [ 1.2911412 , -1.176012  ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 1.5925817489624023, 'test_accuracy': 0.5099396521121761, 'test_f1': 0.5086377927804081, 'test_precision': 0.6257395333445623, 'test_recall': 0.6408924965079816, 'test_runtime': 1483.3298, 'test_samples_per_second': 7.596, 'test_steps_per_second': 0.119})

### **T_HSAB Testing** 

In [ ]:
df_test= pd.read_csv('/content/T_HSAB.csv')
df_test= df_test[['Text','Label']]
df_test

,Text,Label
0,اخس خوي اللي مايخليك تأكل الين يجي أسلوب يهود,1
1,الله أكبر \nالموت لأمريكا\nالموت لاسرائيل\nالل...,1
2,تحميل كتاب الملة والنحلة في اليهودية والمسيحية...,0
3,والله المفروض اول شي نقاطعه البحرين ودبي مع ...,1
4,منذ أن سحب الله البساط من تحت أقدام اليهود بال...,1
...,...,...
1261,لإخواني من أهل السنة الخلص أبشركم .\nثلاث أماك...,0
1262,قاتل اهل السنة في مصر https://t.co/zkrCG5ZLFt,0
1263,حسبي الله لا إله إلا هو عليه توكلت وهو ربّ الع...,0
1264,اتَّبِعِ الْحَقَّ وَلَا تُبَالِ ، وَاحْبِبِ ال...,0


In [ ]:
df_test['Label'].value_counts()

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 1266
  Batch size = 64


PredictionOutput(predictions=array([[ 2.4716668 , -2.3631196 ],
       [ 0.61444694, -0.8683536 ],
       [ 0.5383555 ,  0.04989015],
       ...,
       [ 0.46815807, -0.6665193 ],
       [-0.04376122, -0.20044488],
       [ 0.9072249 , -1.0376639 ]], dtype=float32), label_ids=array([1, 1, 0, ..., 0, 0, 1]), metrics={'test_loss': 1.3681827783584595, 'test_accuracy': 0.47788309636650866, 'test_f1': 0.38093367511124426, 'test_precision': 0.3740690335895672, 'test_recall': 0.41187175066000686, 'test_runtime': 185.8721, 'test_samples_per_second': 6.811, 'test_steps_per_second': 0.108})

### **Mubarak_2020 Testing** 

In [ ]:
df_test= pd.read_csv('/content/Mubarak_2020.csv')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 1100
  Batch size = 64


PredictionOutput(predictions=array([[ 0.8253825 , -0.38989404],
       [ 1.9066529 , -1.8844366 ],
       [ 1.2782637 , -1.0072782 ],
       ...,
       [ 2.1451478 , -1.9099622 ],
       [ 2.936368  , -2.6270804 ],
       [ 2.902935  , -2.7984138 ]], dtype=float32), label_ids=array([1, 0, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.8857645392417908, 'test_accuracy': 0.6863636363636364, 'test_f1': 0.6236608723332728, 'test_precision': 0.7200665188470066, 'test_recall': 0.6340982834682743, 'test_runtime': 145.146, 'test_samples_per_second': 7.579, 'test_steps_per_second': 0.124})

### **Chowdhury Testing** 

In [ ]:
df_test= pd.read_csv('/content/Chowdhury.csv')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

### **Let_Mi Testing** 

In [ ]:
df_test= pd.read_csv('/content/Let_Mi.csv')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

### **ArSarcasm Testing** 

In [ ]:
df_test= pd.read_csv('/content/ArSarcasm')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

### **Mubarak_2022 Testing** 

In [ ]:
df_test= pd.read_csv('/content/Mubarak_2022')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

1    3325
0     675
Name: Label, dtype: int64

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)

### **ArCybC Testing** 

In [ ]:
df_test= pd.read_csv('/content/ArCybC')
df_test= df_test[['Text','Label']]
df_test

In [ ]:
df_test['Label'].value_counts()

In [ ]:
test_texts, test_labels = df_test['Text'],df_test['Label']

In [ ]:
test_encodings = tokenizer(test_texts.to_list(), truncation=True, padding=True, max_length=max_seq_len)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
trainer.predict(test_dataset)